In [50]:
import time
from datetime import datetime

import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, explained_variance_score
from sklearn.metrics import mean_squared_log_error, mean_poisson_deviance, mean_gamma_deviance, mean_tweedie_deviance, max_error

%matplotlib inline

df = pd.read_csv("data/data_test.csv")



In [52]:
df[df['Date'] == '2021-05-10']

,Date,Close,High,Low,Open,Volume,trend_ema_fast,target,pred,error,bool,p_bool,right,name,target_name,days_ahead,day_predict,trend_ema_slow,ema_5
2,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,57.938999,58.076010,57.446632,0.629378,1,0,False,XOM_Q5R_TAH_0,trend_ema_fast,1,2021-05-11,NaN,NaN
102,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,57.938999,58.240440,58.555843,0.315403,1,1,True,XOM_Q5R_TAH_1,trend_ema_fast,2,2021-05-11,NaN,NaN
202,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,57.938999,58.267423,57.826645,0.440779,1,0,False,XOM_Q5R_TAH_2,trend_ema_fast,3,2021-05-11,NaN,NaN
302,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,57.938999,58.513038,59.411556,0.898518,1,1,True,XOM_Q5R_TAH_3,trend_ema_fast,4,2021-05-11,NaN,NaN
402,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,NaN,56.726801,56.651199,0.075601,1,1,True,XOM_Q6R_TAH_0,trend_ema_slow,1,2021-05-11,56.558579,NaN
502,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,NaN,56.905912,56.262836,0.643075,1,0,False,XOM_Q6R_TAH_1,trend_ema_slow,2,2021-05-11,56.558579,NaN
602,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,NaN,57.017758,57.143673,0.125915,1,1,True,XOM_Q6R_TAH_2,trend_ema_slow,3,2021-05-11,56.558579,NaN
702,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,NaN,57.228585,57.134945,0.093640,1,1,True,XOM_Q6R_TAH_3,trend_ema_slow,4,2021-05-11,56.558579,NaN
802,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,NaN,59.346674,59.974174,0.627501,0,1,False,XOM_Q7R_TAH_0,ema_5,1,2021-05-11,NaN,59.605225
902,2021-05-10,60.761753,62.159909,60.713203,61.402577,31956600,NaN,59.279383,59.505795,0.226411,0,0,True,XOM_Q7R_TAH_1,ema_5,2,2021-05-11,NaN,59.605225


In [ ]:
plt.figure(figsize=(13,7))
plt.xticks(rotation=45) 

def plot_graph_market():
    for idx, val in df.iterrows():
        plt.plot([idx, idx], [val['Low'], val['High']], color='Black')


def plot_graph_real(col, color):
    plt.plot(df.index, df[col], color=color)
    plt.scatter(df.index, df[col], color=color)

def plot_graph_pred(index):
    x, y = [], []
    target = []
    color = []
    df_1 = df[df['days_ahead'] == index]
    [y.append(df_1['day_predict'].values[0][:10]) for i in range(3)]
    for i, j in zip(['red', 'blue', 'green'], range(3)):
        target.append(df_1['target'].values[j])
        x.append(df_1['pred'].values[j])
        color.append(i)
    plt.scatter(y, x, c=color, linewidths=4)
    # plt.scatter(y, target, c=color, linewidths=1)
    
targets_name = df['target_name'].drop_duplicates().values
colors = ['red', 'blue', 'green']

plot_graph_market()
plot_graph_real('Close', 'Black')
[plot_graph_real(i, j) for i, j in zip(targets_name, colors)]
[plot_graph_pred(i + 1) for i in range(4)]

plt.grid(True)
plt.show()


In [48]:
best = []

date = df['Date'].drop_duplicates()
for j in range(len(date)):
    ax_df = df[df['Date'] == date[j]]
    ax_best = ax_df.sort_values(['days_ahead', 'error'])
    for i in range(0, 11, 3):
        best.append(ax_best.iloc[i:i+1, 13:14].values[0][0][5])

best = pd.Series(best)
print(best.value_counts(normalize=True))
print(best.value_counts())